# Automated Seagrass Classification Using Earth Engine Python API

This script classify seagrass beds in selected BOA images using ground-data to train three machine learning classifiers: CART, Support Vector Machine and Random Forest. The outputs can be exported to EE Assets. All the training and validation matrices and accuracies can be saved as an Excel file in your working directory.<br/>
**NOTE:** The input image needs to have the bands B1, B2, B3, B4, B5, B8, B11, B12 to apply masks correctly. The classifications will use only the bands B1, B2, B3, B4 and B2B3, which represent the bands that penetrates the most into the water column.

Script by: Luis Lizcano-Sandoval<br/>
College of Marine Sciences, University of South Florida<br/>
Updated: 21/10/2020

<font size="4">**Workflow:**</font>

1. Import required images, collections, data, etc.
2. Mask clouds, land, and deep areas >20m
3. Apply Depth-Invariant Index (generates band-ratios B1B2, B1B3, B2B3)
4. Sample bands: B1, B2, B3, B4, B2B3
5. Train models and classify (CART, SVM and RF)
6. Get confusion matrices and accuracies
7. Export output to EE Assets (.tiff)
8. Save matrices in local computer (.xlxs)

In [ ]:
import ee
import pandas as pd
import xlsxwriter
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from functions import CloudScore6S,landMaskFunction,div
from IPython.display import display, Image

ee.Initialize()

In [2]:
imageID = '20190117T161619_20190117T162429_T17RLM'

<font size="4">**1. Import files:**</font>

Import BOA Images:

In [199]:
## Image Collection
collection = ee.ImageCollection("users/lizcanosandoval/BOA/Sentinel/FL_19")

## Filter collection by image ID:
imageTarget = collection.filter(ee.Filter.eq('file_id',imageID)).first()

## more settings and metadata
imageGeometry = imageTarget.geometry() #Tile geometry.
imageSat = imageTarget.get('satellite').getInfo() #Image satellite
imageTile = imageTarget.get('tile_id').getInfo() #Image tile id
imageDate = imageTarget.get('date').getInfo() #Image date
print('Satellite: ',imageSat)
print('Tile: ',imageTile)
print('Date: ',imageDate)

Satellite:  Sentinel-2B
Tile:  17RLM
Date:  2019-01-17


Visualize image

In [4]:
rgb = ['B4','B3','B2']

# Setting for displaying images:
def displaySettings(image, channels):
    img = Image(url=image.select(channels).getThumbUrl({
        'dimensions': '500x500',
        'min':0,
        'max':0.2,
        'gamma':1.5
        }))
    return display(img)

## Display image:
originalImage = displaySettings(imageTarget, rgb)

Import Ground-Points [Training and Validation] and sandy areas:

In [5]:
# Sandy areas
sand_areas = ee.FeatureCollection("users/lizcanosandoval/Sand"),

# Ground-Points
springsCoast = ee.FeatureCollection("users/lizcanosandoval/ground-points/springs_coast-19"),
springsCoast_noDen = ee.FeatureCollection("users/lizcanosandoval/ground-points/springs_coast-19-noDensities"),
springsCoast_tra = ee.FeatureCollection("users/lizcanosandoval/ground-points/springs_coast-19_tra"),
springsCoast_val = ee.FeatureCollection("users/lizcanosandoval/ground-points/springs_coast-19_val"),

Import other collections:

In [6]:
## ETOPO1: Global 1 Arc-Minute Elevation:
etopo = ee.Image("NOAA/NGDC/ETOPO1")
etopo = etopo.select('bedrock') ## Select the bathymetry band

## Florida Bathymetry [NOAA-90m res]
bathymetry = ee.Image("users/lizcanosandoval/Bathymetry_FL")

## Florida Land [from GADM-HiRes]
gadm_FL = ee.FeatureCollection("users/lizcanosandoval/gadm36_FL")

## Florida's Seagrass Habitats (FWC database)
seagrass = ee.FeatureCollection("users/lizcanosandoval/Seagrass_Habitat_Florida"),

## Sentinel-2 Tiles over seagrass areas:
tiles = ee.FeatureCollection("users/lizcanosandoval/S2_tiles_surf-to-shallow_line")

<font size="3">**Visualization parameters:**</font>

In [7]:
## Palette:
palette = ['030d81','0519ff','05e8ff','11ff01','fbff01','ff9901','ff0000','ad0000']
## RGB
rgbVis = {\
          'min': 0,
          'max': 0.2,
          'gamma': 2,
          'bands': ['B4','B3','B2'],
         }

## Turbidity palette
visTur = {\
         'bands': ['B5'],
         'min': 0.01,
         'max': 0.05,
         'palette': palette,
         }

## Bathymetry palette
visBath = {\
          'min': -20,
          'max': 0,
          'palette': palette,
          }

<font size="4">**Prepare and mask bathymetry data:**</font>

In [8]:
## Mask depth ranges from the Etopo collection [from -25 to 1 meter]
etopo_masked = ee.Image(etopo).updateMask(etopo.lt(1).And(etopo.gt(-25)))

## Resample ETOPO
etopo_resample = etopo_masked.reproject(**{\
                                         'crs': 'EPSG:32617',
                                         'scale': 90,
                                        })
    
## Define a boxcar or low-pass kernel.
kernel = ee.Kernel.square(**{\
                           'radius': 3, 
                           'units': 'pixels', 
                           'normalize': True,
                          })

## Smooth raster
#etopo_kernel = etopo_resample.resample('bilinear')
etopo_kernel = etopo_resample.convolve(kernel)

## Mask the FL bathymetry collection (NOAA):
## Mask depth ranges from the FL bathymetry collection [Mask depth range from -20 to 2 meters]
bathy_masked = ee.Image(bathymetry).updateMask(bathymetry.lt(2).And(bathymetry.gt(-25)))

Clip bathymetry to the tile geometry/bounds:

In [9]:
## Clip bathymetry layers to tile geometry
bathyBand = bathy_masked.clip(imageGeometry) ##Florida bathymetry clip (90m res)
etopo_clip = etopo_kernel.clip(imageGeometry) ##Etopo clip + kernel (90m resampled)

As the bathymetry dataset from NOAA has small gaps in some areas we need to fill them using the resampled ETOPO collection:

In [10]:
## Vectorize (to FeatureCollection of polygons) the bathymetry collection,
## which is a raster image.
bathyVector = bathyBand.toByte().reduceToVectors(**{
  'reducer': ee.Reducer.countEvery(),
  'crs': 'EPSG:4326',
  'geometry': None,
  'eightConnected': False,
  'labelProperty': 'bathymetry',
  'scale': 1000,
  'geometryType': 'polygon',
  'maxPixels': 1e9
})

## Extract the bathymetry values from the Etopo layer corresponding to the gap 
## found in the FL Bathymetry Data.
bathyOutside = ee.Image.constant(1).clip(bathyVector).mask().Not()
bathyGap = etopo_clip.updateMask(bathyOutside)
bathyVectorGap = bathyGap.toByte().reduceToVectors(**{
  'reducer': ee.Reducer.countEvery(),
  'crs': 'EPSG:4326',
  'geometry': None,
  'eightConnected': False,
  'labelProperty': 'bathymetry',
  'scale': 1000,
  'geometryType': 'polygon',
  'maxPixels': 1e9
});
bathyVectorGap = bathyVectorGap.geometry().buffer(50) ##Fill gap and extend a buffer of 50m
bathyOutside2 = ee.Image.constant(1).clip(bathyVectorGap).mask()
bathyGap2 = etopo_clip.updateMask(bathyOutside2) ##Clip the gap in the ETOPO collection.

#Finally, fill the gap in the NOAA bathymetry dataset. This is the new NOAA's bathymetry raster layer:
bathyFilled = ee.ImageCollection([bathyBand,bathyGap2.select(['bedrock'],['b1'])]).mosaic()

<font size="4">**2. Apply masks to image:**</font>

Cloud mask:

In [11]:
## Threshold value set to 2
cloudMask = CloudScore6S(imageTarget, 2)

## Display image:
#cloudImage = displaySettings(cloudMask, rgb)

Land mask:

In [12]:
landMask = landMaskFunction(cloudMask, gadm_FL)

## Display image:
#landImage = displaySettings(landMask, rgb)

Bathymetry mask:

In [13]:
bathyMask = landMask.clip(bathyVector) ##Using the NOAA dataset.

## Display image:
#bathyImage = displaySettings(bathyMask, rgb)

<font size="4">**3. Water column correction:**</font>

In [35]:
## Filter sand polygons by tile/area:
#sand = ee.FeatureCollection(sand_areas).filter(ee.Filter.eq('t_sentinel',tile))
sand = ee.FeatureCollection(sand_areas).flatten().filterBounds(imageGeometry)
print('Number of sand polygons: ',sand.size().getInfo())

## Run the Depth-Invariant Index Function
bands = ['B1','B2','B3'] #Select bands
imageDIV = div(bathyMask, bands, sand)
#print(imageDIV.getInfo())

## Display one of the bands ['B1B2', 'B1B3', 'B2B3']
# imgDIV = Image(url=imageDIV.select('B2B3').getThumbUrl({
#     'dimensions': '500x500',
#     'min':-3,
#     'max':-1.5,
#     'gamma':1.5
#     }))
# display(imgDIV)

Number of sand polygons:  6


<font size="4">**4. Sampling Data:**</font>

Classes are:
* 0: Softbottom
* 1: Hardbottom
* 2: Seagrass
* 3: Sparse seagrass //if available

In [47]:
## Collection with classes
classNames = ee.FeatureCollection(springsCoast).flatten()
print('Classes: ', classNames.aggregate_array('class').distinct().getInfo())

## Add training and validation datasets separately.
classTraining = ee.FeatureCollection(springsCoast_tra).flatten()
classValidation = ee.FeatureCollection(springsCoast_val).flatten()
numberClasses = classNames.aggregate_array('class').distinct().length()
print('Ground Points per Class:', classNames.aggregate_histogram('class').getInfo())

Classes:  [0, 1, 2, 3]
Ground Points per Class: {'0': 971, '1': 300, '2': 800, '3': 192}


Select the bands to sample:

In [44]:
## Select bands to sample
bandsClass = ['B1','B2', 'B3', 'B4','B2B3']

## Add bands of interest to sample training points:
imageClassify = bathyMask.addBands(imageDIV.select(['B2B3'])).select(bandsClass)
print('Bands to sample:',imageClassify.bandNames().getInfo())
##Other configurations:
#imageClassify = imageDIV.select(bandsClass); // Depth invariant Bands B1B2,B1B3,B2B3.
#imageClassify = imageKD.select(bandsClass)//.addBands(bathyFilled); //KD corr B1-B4, with or without bathymetry.

## Define a boxcar or low-pass kernel (Used if want to smooth the image)
smooth = ee.Kernel.euclidean(**{
    'radius': 1, 
    'units': 'pixels', 
    'normalize': True
})
#imageClassify = imageClassify.convolve(smooth)

Bands to sample: ['B1', 'B2', 'B3', 'B4', 'B2B3']


Sample training and validation data:

In [48]:
## Sample multi-spectral data using all ground points.
trainingData = imageClassify.sampleRegions(**{
    'collection': classTraining,
    'properties': ['class'],
    'scale': 10})
print('Training size (70%):',trainingData.size().getInfo())

validationData = imageClassify.sampleRegions(**{
    'collection': classValidation,
    'properties': ['class'],
    'scale': 10})

Training size (70%): 1574


<font size="4">**5. Train models and Classify:**</font>

In [49]:
## Train CART classifier
trainCART = ee.Classifier.smileCart().train(trainingData,'class',bandsClass)

## Train SVM classifier
SVM = ee.Classifier.libsvm(**{
   'kernelType': 'RBF',
   'gamma': 100,
   'cost': 100
})
trainSVM = SVM.train(**{
   'features': trainingData,
   'classProperty': 'class',
   'inputProperties': bandsClass
})

## Train RF classifier
trainRF = ee.Classifier.smileRandomForest(20).train(trainingData, 'class', bandsClass)

#### Classify the image using the trained classifier
classifiedCART = imageClassify.classify(trainCART)
classifiedSVM = imageClassify.classify(trainSVM)
classifiedRF = imageClassify.classify(trainRF)

Display classified images:

In [50]:
## Define a palette for the distinct classes
classPalette = ['#3090C7','#CD7F32','#004E00','#78F878']

imgSVM = Image(url=classifiedSVM.getThumbUrl({
    'dimensions': '500x500',
    'min':0,
    'max':3,
    'palette': classPalette
    }))
display(imgSVM)

<font size="4">**6. Get accuracy matrices:**</font>

Training accuracies:

In [61]:
## Get a confusion matrix representing resubstitution accuracy.
## {Resubstitution error is the error of a model on the training data.}
## Axis 0 (first level) of the matrix correspond to the input classes (columns), 
## and axis 1 (second level) to the output classes (rows).
matrixTrainingCART = trainCART.confusionMatrix()
matrixTrainingSVM = trainSVM.confusionMatrix()
matrixTrainingRF = trainRF.confusionMatrix()

#print('CART Training Confusion Matrix: ', matrixTrainingCART.getInfo())
#print('SVM Training Confusion Matrix: ', matrixTrainingSVM.getInfo())
#print('RF Training Confusion Matrix: ', matrixTrainingRF.getInfo())

print('Cart Training overall accuracy: ', round((matrixTrainingCART.accuracy().getInfo()), 4))
print('SVM Training overall accuracy: ', round((matrixTrainingSVM.accuracy().getInfo()), 4))
print('RF Training overall accuracy: ', round((matrixTrainingRF.accuracy().getInfo()), 4))

Cart Training overall accuracy:  0.9987
SVM Training overall accuracy:  0.8577
RF Training overall accuracy:  0.9759


Validation accuracies:

In [71]:
## Calculate accuracy using validation data
## Classify the image using the trained classifier
validationCART = validationData.classify(trainCART)
validationSVM = validationData.classify(trainSVM)
validationRF = validationData.classify(trainRF)

## Get a confusion matrix representing expected accuracy (Using validation points - 30%), where:
#  0: Softbottom
#  1: Hardbottom
#  2: Dense Seagrass
#  3: Spare Seagrass

## Axis 0 (the rows) of the matrix correspond to the actual values, 
## and Axis 1 (the columns) to the predicted values.
errorMx = {'actual': 'class', 'predicted': 'classification'}
errorMatrixCART = validationCART.errorMatrix(**errorMx)
errorMatrixSVM = validationSVM.errorMatrix(**errorMx)
errorMatrixRF = validationRF.errorMatrix(**errorMx)

#print('CART Validation Error Matrix: ', errorMatrixCART.getInfo())
#print('SVM Validation Error Matrix: ', errorMatrixSVM.getInfo())
#print('RF Validation Error Matrix: ', errorMatrixRF.getInfo())

print('CART validation overall accuracy: ', round((errorMatrixCART.accuracy().getInfo()), 4))
print('SVM validation overall accuracy: ', round((errorMatrixSVM.accuracy().getInfo()), 4))
print('RF Validation overall accuracy: ', round((errorMatrixRF.accuracy().getInfo()), 4))

CART validation overall accuracy:  0.8292
SVM validation overall accuracy:  0.8234
RF Validation overall accuracy:  0.8613


User and Producer accuracies:

In [81]:
## Estimate user and producer accuracies
producerAccuracyCART = errorMatrixCART.producersAccuracy()
producerAccuracySVM = errorMatrixSVM.producersAccuracy()
producerAccuracyRF = errorMatrixRF.producersAccuracy()

userAccuracyCART = errorMatrixCART.consumersAccuracy()
userAccuracySVM = errorMatrixSVM.consumersAccuracy()
userAccuracyRF = errorMatrixRF.consumersAccuracy()

#print('Producer Accuracy CART: ',producerAccuracyCART.getInfo())
#print('Producer Accuracy SVM: ',producerAccuracySVM.getInfo())
#print('Producer Accuracy RF: ',producerAccuracyRF.getInfo())
#print('User Accuracy CART: ',userAccuracyCART.getInfo())
#print('User Accuracy SVM: ',userAccuracySVM.getInfo())
#print('User Accuracy RF: ',userAccuracyRF.getInfo())

Producer Accuracy SVM:  [[0.86], [0.7261904761904762], [0.8724279835390947], [0.5689655172413793]]
User Accuracy SVM:  [[0.821656050955414, 0.6931818181818182, 0.8760330578512396, 0.8048780487804879]]


Print accuracies as Pandas format:

In [189]:
## Create a pandas dataframe with producer and user accuracies:
rowIndex = {0:'Sb', 1:'Hb', 2:'Dn', 3:'Sp'}
dfPA_CART = pd.DataFrame(producerAccuracyCART.getInfo(), columns=['Producer'])
dfUA_CART = pd.DataFrame(userAccuracyCART.getInfo()).transpose()
dfPA_SVM = pd.DataFrame(producerAccuracySVM.getInfo(), columns=['Producer'])
dfUA_SVM = pd.DataFrame(userAccuracySVM.getInfo()).transpose()
dfPA_RF = pd.DataFrame(producerAccuracyRF.getInfo(), columns=['Producer'])
dfUA_RF = pd.DataFrame(userAccuracyRF.getInfo()).transpose()

PU_CART = pd.concat([dfPA_CART, dfUA_CART.rename(columns={0:'User'})], axis=1).rename(index=rowIndex)
PU_SVM = pd.concat([dfPA_SVM, dfUA_SVM.rename(columns={0:'User'})], axis=1).rename(index=rowIndex)
PU_RF = pd.concat([dfPA_RF, dfUA_RF.rename(columns={0:'User'})], axis=1).rename(index=rowIndex)

print(' CART: \n', PU_CART)
print('\n SVM: \n', PU_SVM)
print('\n RF: \n', PU_RF)

 CART: 
     Producer      User
Sb  0.830000  0.849829
Hb  0.821429  0.711340
Dn  0.876543  0.880165
Sp  0.637931  0.698113

 SVM: 
     Producer      User
Sb  0.860000  0.821656
Hb  0.726190  0.693182
Dn  0.872428  0.876033
Sp  0.568966  0.804878

 RF: 
     Producer      User
Sb  0.900000  0.859873
Hb  0.773810  0.792683
Dn  0.909465  0.894737
Sp  0.586207  0.809524


Kappa coefficients:

In [78]:
# The Kappa Coefficient is generated from a statistical test to evaluate the accuracy 
# of a classification. Kappa essentially evaluate how well the classification performed 
# as compared to just randomly assigning values, i.e. did the classification do better 
# than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that 
# the classification is no better than a random classification. A negative number 
# indicates the classification is significantly worse than random. A value close to 1 
# indicates that the classification is significantly better than random.

kappaCART = errorMatrixCART.kappa()
kappaSVM = errorMatrixSVM.kappa()
kappaRF = errorMatrixRF.kappa()

print('Kappa CART: ', round((kappaCART.getInfo()), 4))
print('Kappa SVM: ', round((kappaSVM.getInfo()), 4))
print('Kappa RF: ', round((kappaRF.getInfo()), 4))

Kappa CART:  0.7425
Kappa SVM:  0.7295
Kappa RF:  0.7871


<font size="4">**7. Export Classified Images:**</font>

In [206]:
# Set the scale properly
scale = []
sat = []
method = ['CART','SVM','RF']
classifiedCollection = ee.ImageCollection([classifiedCART,classifiedSVM,classifiedRF])
classifiedList = classifiedCollection.toList(classifiedCollection.size())
classifiedSize = classifiedList.size().getInfo()
print('Wait for submission')

for i in range(classifiedSize):
    if 'Sentinel' in imageSat:
        sat = 'Sentinel'
        scale = 10 #For Sentinel
    else:
        sat = 'Landsat'
        scale = 30 #For Landsat 
        
    ## Select image
    image = ee.Image(classifiedList.get(i))
    
    # set some properties for export
    output = image.set({'satellite': imageSat,
                   'tile_id': imageTile,
                   'file_id': imageID,                                               
                   'date': imageDate,
                   'classifier': method[i],
                   'generator': 'Lizcano-Sandoval',
                        })

    # define YOUR assetID. (This do not create folders, you need to create them manually)
    assetID = 'users/lizcanosandoval/Seagrass/'+sat+'/'+'FL_19/' ##This goes to an ImageCollection folder
    fileName = imageTile+'_'+imageID+'_'+ method[i]
    path = assetID + fileName
    
    ## Batch Export to Assets
    ee.batch.Export.image.toAsset(\
        image = ee.Image(output),                                                    
        description = method[i]+'_'+imageID,
        assetId = path,
        region = imageGeometry.buffer(10),                                      
        maxPixels = 1e13,
        scale = scale).start()
    print('Classified Image '+str(i+1)+': '+imageID+'_'+ method[i]+' submitted...')
print('All images submitted!')

Wait for submission
Classified Image 1: 20190117T161619_20190117T162429_T17RLM_CART submitted...
Classified Image 2: 20190117T161619_20190117T162429_T17RLM_SVM submitted...
Classified Image 3: 20190117T161619_20190117T162429_T17RLM_RF submitted...
All images submitted!


<font size="4">**8. Save Matrices to Working Directory:**</font>

Extract values from each matrix

In [198]:
CART_trainingMatrix = matrixTrainingCART.array().getInfo()
CART_trainingAccuracy = matrixTrainingCART.accuracy().getInfo()
CART_errorMatrix = errorMatrixCART.array().getInfo()
CART_errorAccuracy = errorMatrixCART.accuracy().getInfo()
CART_producerAccuracy = producerAccuracyCART.getInfo()
CART_userAccuracy = userAccuracyCART.getInfo()
CART_kappa = kappaCART.getInfo()
SVM_trainingMatrix = matrixTrainingSVM.array().getInfo()
SVM_trainingAccuracy = matrixTrainingSVM.accuracy().getInfo()
SVM_errorMatrix = errorMatrixSVM.array().getInfo()
SVM_errorAccuracy = errorMatrixSVM.accuracy().getInfo()
SVM_producerAccuracy = producerAccuracySVM.getInfo()
SVM_userAccuracy = userAccuracySVM.getInfo()
SVM_kappa = kappaSVM.getInfo()
RF_trainingMatrix = matrixTrainingRF.array().getInfo()
RF_trainingAccuracy = matrixTrainingRF.accuracy().getInfo()
RF_errorMatrix = errorMatrixRF.array().getInfo()
RF_errorAccuracy = errorMatrixRF.accuracy().getInfo()
RF_producerAccuracy = producerAccuracyRF.getInfo()
RF_userAccuracy = userAccuracyRF.getInfo()
RF_kappa = kappaRF.getInfo()

print('These matrices needs to be reformatted, e.g.: ', CART_trainingMatrix)

This matrices needs to be reformatted:  [[667, 0, 0, 0], [1, 215, 0, 0], [1, 0, 556, 0], [0, 0, 0, 134]]


Convert matrices to pandas dataframes

In [193]:
#Training Matrices
TM_CART = pd.DataFrame(CART_trainingMatrix).rename(columns=rowIndex, index=rowIndex)
TM_SVM = pd.DataFrame(SVM_trainingMatrix).rename(columns=rowIndex, index=rowIndex)
TM_RF = pd.DataFrame(RF_trainingMatrix).rename(columns=rowIndex, index=rowIndex)
TM_concat = pd.concat([TM_CART, TM_SVM, TM_RF], keys=['CART','SVM','RF'])

#Training Accuracies
TA_CART = pd.Series(CART_trainingAccuracy)
TA_SVM = pd.Series(SVM_trainingAccuracy)
TA_RF = pd.Series(RF_trainingAccuracy)
TA_concat = pd.DataFrame(pd.concat([TA_CART, TA_SVM, TA_RF],ignore_index=True), columns=(['Tr_Accuracy']))\
                .rename({0:'CART',1:'SVM',2:'RF'})

#Validation-Error Matrices
VM_CART = pd.DataFrame(CART_errorMatrix).rename(columns=rowIndex, index=rowIndex)
VM_SVM = pd.DataFrame(SVM_errorMatrix).rename(columns=rowIndex, index=rowIndex)
VM_RF = pd.DataFrame(RF_errorMatrix).rename(columns=rowIndex, index=rowIndex)
VM_concat = pd.concat([VM_CART, VM_SVM, VM_RF], keys=['CART','SVM','RF'])

#Validation Accuracies
VA_CART = pd.Series(CART_errorAccuracy)
VA_SVM = pd.Series(SVM_errorAccuracy)
VA_RF = pd.Series(RF_errorAccuracy)
VA_concat = pd.DataFrame(pd.concat([VA_CART, VA_SVM, VA_RF],ignore_index=True), columns=(['Va_Accuracy']))\
                .rename({0:'CART',1:'SVM',2:'RF'})

#Producer-User Accuracies
PU_concat = pd.concat([PU_CART, PU_SVM, PU_RF], keys=['CART','SVM','RF'])

#Kappa coefficients
Kp_CART = pd.Series(CART_kappa)
Kp_SVM = pd.Series(SVM_kappa)
Kp_RF = pd.Series(RF_kappa)
Kp_concat = pd.DataFrame(pd.concat([Kp_CART, Kp_SVM, Kp_RF],ignore_index=True), columns=(['Kappa']))\
                .rename({0:'CART',1:'SVM',2:'RF'})

Kp_concat.style.set_caption('Kappa coefficients')
#print(Kp_concat)

,Kappa
CART,0.742545
SVM,0.729531
RF,0.787114


Organize each matrix in separate excel sheets

In [208]:
excelName = 'Mrx_'+imageID+'.xlsx'
excel = pd.ExcelWriter(excelName, engine='xlsxwriter')

TM_concat.to_excel(excel, sheet_name='TrMrx', index=True, startrow=0)
TA_concat.to_excel(excel, sheet_name='TrAcc', index=True, startrow=0)
VM_concat.to_excel(excel, sheet_name='VaMrx', index=True, startrow=0)
VA_concat.to_excel(excel, sheet_name='VaAcc', index=True, startrow=0)
PU_concat.to_excel(excel, sheet_name='PU-Mrx', index=True, startrow=0)
Kp_concat.to_excel(excel, sheet_name='Kappa', index=True, startrow=0)

Save matrices as .xlsx file:

In [209]:
excel.save()